<a href="https://colab.research.google.com/github/Andy14py/my_projects/blob/main/DataScience_Project_Limpieza_data_Mundial_football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> **PREDECIR CAMPEON DEL MUNDIAL QATAR** <center>

# FASE 2: Limpieza y transformacion de datos
luego de la Fase 1 procederemos a transformar y limpiar los datos extraidos para poder procesarlos en nuestro modelo predictivo posterior

In [1]:
import pandas as pd 
df_data_historica=pd.read_csv("fifa_worldcup_historical_data.csv")
df_fixture=pd.read_csv("fifa_worldcup_fixture.csv")
df_data_faltante=pd.read_csv("fifa_worldcup_missing_data.csv")

In [2]:
# Limpiando columnas Home y away de espacios vacios con metodo strip()
df_fixture["home"]=df_fixture["home"].str.strip()
df_fixture["away"]=df_fixture["away"].str.strip()

In [3]:
#Limpiando df_data_faltante y concatenarlo con df_data_historica
#df_data_faltante[df_data_faltante['home'].isnull()] verificando si hay data faltante
df_data_faltante.dropna(inplace=True)
df_data_historica=pd.concat([df_data_faltante,df_data_historica],ignore_index=True)
#eliminando duplicados
df_data_historica.drop_duplicates(inplace=True)
df_data_historica.sort_values("year",inplace=True)
df_fixture

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


Limpiando df_data_historica nueva

In [ ]:
# buscamos los subniveles en score e identificamos valores erroneos como w/o[a] y filas con (a.e.t.)
d=df_data_historica["score"].unique()
# limpiamos las filas que contengan w/o[a] ,localizamos las filas 
filas_eliminar = df_data_historica.loc[df_data_historica.eq("w/o[a]").any(axis=1)]
df_data_historica = df_data_historica.drop(filas_eliminar.index)

# Eliminamos espacios vacios en la columna home y away y score
df_data_historica["home"]=df_data_historica["home"].str.strip()
df_data_historica["away"]=df_data_historica["away"].str.strip()
df_data_historica['score']=df_data_historica['score'].str.strip()
# Eliminamos registros duplicados y ordenamos por año
df_data_historica.drop_duplicates(inplace=True)
df_data_historica.sort_values('year',inplace=True)
# Reseteamos el index actual y eliminamos el index anterior drop=True
df_data_historica=df_data_historica.reset_index(drop=True)
df_data_historica
# Modificando registros con (a.e.t.) en sus valores la expresion [^\d–] 
#nos devuelve todos los registros que no contentgan un digito(d) no un guion(-)
#df_data_historica[df_data_historica['score'].str.contains('[^\d–]')]
df_data_historica['score']=df_data_historica['score'].str.replace('[^\d–]','',regex=True)
df_data_historica
# Separando los goles(score) y asignandolos a los equipos home y away
df_data_historica[['HomeGoals','AwayGoals']]=df_data_historica['score'].str.split('–', expand=True)

In [ ]:
# Eliminamos fila score axis= 0(fila) axis=1(columna)
df_data_historica.drop('score',axis=1)
# renombrar columnas con metodo rename
df_data_historica.rename(columns={'home':'HomeTeam','away':'AwayTeam','year':'Year'}, inplace=True)
df_data_historica

,HomeTeam,score,AwayTeam,Year,HomeGoals,AwayGoals
0,France,4–1,Mexico,1930,4,1
1,Uruguay,6–1,Yugoslavia,1930,6,1
2,Uruguay,4–2,Argentina,1930,4,2
3,Argentina,6–1,United States,1930,6,1
4,Paraguay,1–0,Belgium,1930,1,0
...,...,...,...,...,...,...
895,Egypt,0–1,Uruguay,2018,0,1
896,Russia,5–0,Saudi Arabia,2018,5,0
897,Peru,0–1,Denmark,2018,0,1
898,Germany,0–1,Mexico,2018,0,1


In [ ]:
# verificando tipos de datos, los datos de HomeGoals y AwayGoals tienen tipo object debe ser int
# tener cuidado si en una fila hay valores nulos no hara la conversion
#verificando tipos de datos
df_data_historica.dtypes
#eliminando duplicados
df_data_historica.dropna(inplace=True)
#asignando un tipo de dato distinto a cada columna
df_data_historica=df_data_historica.astype({"HomeGoals":int,'AwayGoals':int})
df_data_historica.dtypes
#Creando una nueva columan que sera la suma de los goles totales
df_data_historica['TotalGoals']=df_data_historica["HomeGoals"]+df_data_historica['AwayGoals']
#df_data_historica.drop('score',axis=1,inplace=True) eliminamos la columna score
df_data_historica

,HomeTeam,score,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,4–1,Mexico,1930,4,1,5
1,Uruguay,6–1,Yugoslavia,1930,6,1,7
2,Uruguay,4–2,Argentina,1930,4,2,6
3,Argentina,6–1,United States,1930,6,1,7
4,Paraguay,1–0,Belgium,1930,1,0,1
...,...,...,...,...,...,...,...
895,Egypt,0–1,Uruguay,2018,0,1,1
896,Russia,5–0,Saudi Arabia,2018,5,0,5
897,Peru,0–1,Denmark,2018,0,1,1
898,Germany,0–1,Mexico,2018,0,1,1


In [ ]:
df_data_historica.to_csv('clean_data_historica_fifa_worldcup.csv', index=False)
df_fixture.to_csv('clean_data_fixture.csv', index=False)

In [ ]:
# verificamos el numero de encuentros por competicion
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

for year in years:
    print(year, len(df_data_historica[df_data_historica['Year']==year]))